# Importes e bibliotecas

In [ ]:
!pip install transformers 
!python -m spacy download pt

     |████████████████████████████████| 21.2 MB 2.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
# importes de bibliotecas 
import spacy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
# modelos pré treinados para fazer a tradução para o inglês 
tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/translation-pt-en-t5")
model = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-pt-en-t5")

In [ ]:
# declarando qual a linguagem que o spacy vai trabalhar 
nlp = spacy.load('pt')
# coding: utf-8

# Raciocínio Baseado em Casos

## Avaliação dos valores dos atributos conforme frase

In [ ]:
def avaliacao_atributos(frase):  
  # lista que vai guardar a relacao dos atributos com a frase 
  lista = []

  # padronizacoes 
  frase_mod = frase.lower()
  doc = nlp(frase_mod)
  frase_mod = [token.orth_ for token in doc] # para se assegurar que as pontuacoes
                                            # terão um index reservado no split()   

  # traducao para os atributos 4 e 5
  pten_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
  trad = pten_pipeline(frase)
  traducao = (trad[0]['generated_text']).lower()

  # Primeiro atributo - Está no final da frase?
  ultima_palavra = frase_mod[len(frase_mod)-2]
  if(ultima_palavra == 'que' or ultima_palavra == 'porque' or ultima_palavra == 'porquê' or ultima_palavra == 'quê'):
    lista.append(1)
  else:
    lista.append(0)

  # Segundo atributo - É antecido por um artigo como “o, os, um, uns”?
  for i in range(0,len(frase_mod)):
    if(frase_mod[i]=='porque' or 
      frase_mod[i]=='porquê' or 
      (frase_mod[i]=='por' and frase_mod[i+1] == 'quê') or 
      (frase_mod[i]=='por' and frase_mod[i+1] == 'que')):
      if(frase_mod[i-1]=='o' or frase_mod[i-1]=='os' or frase_mod[i-1]=='um' or frase_mod[i-1]=='uns'):
        lista.append(1)
      else:
        lista.append(0)

  # Terceiro atributo - É antecedido por um substantivo? 
  morfologia = [token.pos_ for token in doc] # pos = parts of speech
  for i in range(0,len(frase_mod)):
    if(frase_mod[i]=='porque' or 
      frase_mod[i]=='porquê' or 
      (frase_mod[i]=='por' and frase_mod[i+1] == 'quê') or 
      (frase_mod[i]=='por' and frase_mod[i+1] == 'que')):
      if(morfologia[i-1]=='NOUN'):
        lista.append(1)
      else:
        lista.append(0) 

  # Quarto atributo - Pode ser substituído por “pois”?
  if ('because' in traducao):
    lista.append(1)
  else:
    lista.append(0)

  # Quinto atributo - Pode ser substuído por “por qual motivo”?
  if ('why' in traducao):
    lista.append(1)
  else:
    lista.append(0)
  
  return lista

## Criação dos casos bases

In [ ]:
# Caso 1 - Por que (em perguntas)
por_que1 = avaliacao_atributos('João, por que você brigou com sua mãe?')
print('CASO 1 - POR QUE - ', por_que1)

# Caso 2 - Por que (em frases afirmativas)
por_que2 = avaliacao_atributos('Não achei o caminho por que passei.')
print('CASO 2 - POR QUE - ', por_que2)

# Caso 3 - Por quê
por_que3 = avaliacao_atributos('Você não comeu por quê?')
print('CASO 3 - POR QUÊ - ', por_que3)

# Caso 4 - Porquê
porque1 = avaliacao_atributos('Nunca vou lhe dizer o porquê da minha beleza.')
print('CASO 4 - PORQUÊ - ', porque1)

# Caso 5 - Porque
porque2 = avaliacao_atributos('Eu sou feliz porque já fui triste.')
print('CASO 5 - PORQUE - ', porque2)

todos_casos = [por_que1, por_que2, por_que3, porque1, porque2]
casos = ['por que','por que', 'por quê', 'porquê', 'porque']

CASO 1 - POR QUE -  [0, 0, 0, 0, 1]
CASO 2 - POR QUE -  [0, 0, 1, 0, 0]
CASO 3 - POR QUÊ -  [1, 0, 0, 0, 1]
CASO 4 - PORQUÊ -  [0, 1, 0, 0, 0]
CASO 5 - PORQUE -  [0, 0, 0, 1, 0]


## Cálculo das similaridades com o novo caso 

In [ ]:
def calculo_similaridade(lista):
  sim_global = [0.0,0.0,0.0,0.0,0.0]
  for i in range(len(lista)):
    for j in range(len(todos_casos)):
      if(lista[i]==todos_casos[j][i]):
        if(i==0):
          sim_global[j]+=3
        elif(i==1):
          sim_global[j]+=5
        elif(i==2):
          sim_global[j]+=2
        elif(i==3):
          sim_global[j]+=1
        elif(i==4):
          sim_global[j]+=1
  sim_global = [number/12 for number in sim_global]
  #print(sim_global)
  caso = sim_global.index(max(sim_global))
  porque = casos[caso] 
  return porque 

In [ ]:
def palavra_porque(frase):
  frase_mod = frase.lower()
  doc = nlp(frase_mod)
  frase_mod = [token.orth_ for token in doc]
  porque = ''                      
  for i in range(0,len(frase_mod)):
    if(frase_mod[i]=='porque' or
      frase_mod[i]=='porquê'):
      porque = frase_mod[i]
    elif((frase_mod[i]=='por' and frase_mod[i+1] == 'quê') or 
      (frase_mod[i]=='por' and frase_mod[i+1] == 'que')):
      porque = frase_mod[i]+' '+frase_mod[i+1]
  return porque 

## Resultado

In [ ]:
frase = "Pedro, porque você está chorando?"
lista = avaliacao_atributos(frase)
porque = palavra_porque(frase)
caso = calculo_similaridade(lista)
if (caso==porque):
  print("Acertou! O uso dos porquês está correto.")
else:
  print("O uso dos porquês está incorreto. Por favor, substituia o porquê que você digitou pelo '",caso,"'.")

Acertou! O uso dos porquês está correto.


# Teste com qualquer frase

In [ ]:
#@title Escreva a frase que você está com dúvida sobre o uso dos porquês. 
#@markdown Não esqueça da pontuação final!
frase = "Eu te amo porque voc\xEA \xE9 rico. " #@param {type:"string"}

lista = avaliacao_atributos(frase)
porque = palavra_porque(frase)
caso = calculo_similaridade(lista)
# print(lista)
# print(porque)
# print(caso)
if (caso==porque):
  print("Acertou! O uso dos porquês está correto.")
else:
  print("O uso dos porquês está incorreto. Por favor, substituia o porquê que você digitou pelo '",caso,"'.")


Acertou! O uso dos porquês está correto.


# Referências bibliográficas

- LOPES, Alexandre. NOGUEIRA, Rodrigo. LOTUFO, Roberto. PEDRINI, Helio. Universidade Estadual de Campinas (UNICAMP). **Training Strategies for Portuguese-English and English-Portuguese Translation.** Disponível em: https://huggingface.co/unicamp-dl/translation-pt-en-t5. Acesso em 20 de fev. de 2022. 

- PORTELLA, Leticia. **https://leportella.com/pt-br/npl-com-spacy/.** Disponível em: https://leportella.com/pt-br/npl-com-spacy/. Acesso em 20 de fev. de 2022. Acesso em 20 de fev. de 2022. 

- RIBEIRO, Roberto. Universidade Federal de Lavras (UFLA). **USO DO PORQUE, PORQUÊ, POR QUE OU POR QUÊ.** Disponível em: https://www.ufla.br/dcom/2018/05/22/uso-do-porque-porque-por-que-ou-por-que/. Acesso em 20 de fev. de 2022.

